In [1]:
import duckdb
duckdb.sql("ATTACH 'dbname=mimic4 user=szu004' AS db (TYPE POSTGRES, READ_ONLY)")
duckdb.sql("USE db")

In [2]:
# ALL ICU stays (paper reports 765,40)
duckdb.sql("SELECT COUNT(*) as icu_stay_count FROM mimiciv_derived.icustay_detail")

┌────────────────┐
│ icu_stay_count │
│     int64      │
├────────────────┤
│          73181 │
└────────────────┘

In [3]:
duckdb.sql("SELECT COUNT(DISTINCT subject_id) as icu_stay_count FROM mimiciv_derived.icustay_detail")

┌────────────────┐
│ icu_stay_count │
│     int64      │
├────────────────┤
│          50920 │
└────────────────┘

In [4]:
# Ventilation interventions
stays_with_interventions_org = duckdb.sql("""
WITH iws AS (
    SELECT stay_id, starttime as  inttime, ventilation_status AS int_type, row_number() OVER (PARTITION BY stay_id ORDER BY starttime) AS int_sequence
    FROM mimiciv_derived.ventilation
    WHERE ventilation_status NOT in ('None', 'SupplementalOxygen')
)
SELECT stay_id, inttime, int_type FROM iws WHERE int_sequence = 1
""")
stays_with_interventions_org.count("*")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        29367 │
└──────────────┘

In [5]:
# Alternative way to get the first intervention
ventilation_type = duckdb.sql("""
WITH odd AS (
    SELECT subject_id, stay_id, charttime, o2_delivery_device_1 AS o2_delivery_device FROM mimiciv_derived.oxygen_delivery
    UNION SELECT subject_id, stay_id, charttime, o2_delivery_device_2 AS o2_delivery_device FROM mimiciv_derived.oxygen_delivery
    UNION SELECT subject_id, stay_id, charttime, o2_delivery_device_3 AS o2_delivery_device FROM mimiciv_derived.oxygen_delivery
    UNION SELECT subject_id, stay_id, charttime, o2_delivery_device_4 AS o2_delivery_device FROM mimiciv_derived.oxygen_delivery
), odd1 AS (SELECT *, 
    CASE
    -- tracheostomy
    WHEN o2_delivery_device IN (
        'Tracheostomy tube',
        'Trach mask ' -- 16435 observations
        -- 'T-piece', -- 1135 observations (T-piece could be either InvasiveVent or Tracheostomy)
        ) THEN 'Tracheostomy'
    -- mechanical / invasive ventilation
    WHEN o2_delivery_device IN (
        'Endotracheal tube'
        ) THEN 'InvasiveVent'
    -- NIV
    WHEN o2_delivery_device IN (
        'Bipap mask ', -- 8997 observations
        'CPAP mask ' -- 5568 observations
        ) THEN 'NonInvasiveVent'
    -- high flow nasal cannula
    when o2_delivery_device IN (
        'High flow nasal cannula' -- 925 observations
        ) THEN 'HFNC'
    -- non rebreather
    WHEN o2_delivery_device IN ( 
        'Non-rebreather', -- 5182 observations
        'Face tent', -- 24601 observations
        'Aerosol-cool', -- 24560 observations
        'Venti mask ', -- 1947 observations
        'Medium conc mask ', -- 1888 observations
        'Ultrasonic neb', -- 9 observations
        'Vapomist', -- 3 observations
        'Oxymizer', -- 1301 observations
        'High flow neb', -- 10785 observations
        'Nasal cannula'
        ) THEN 'SupplementalOxygen'
    WHEN o2_delivery_device in (
        'None'
     ) THEN 'None'
    -- not categorized: other
    ELSE NULL END AS ventilation_status
FROM odd WHERE o2_delivery_device  IS NOT NULL
)
SELECT subject_id, stay_id, charttime AS starttime, ventilation_status FROM odd1
""")
ventilation_type
stays_with_interventions = duckdb.sql("""
WITH iws AS (
    SELECT stay_id, starttime as  inttime, ventilation_status AS int_type,
row_number() OVER (PARTITION BY stay_id ORDER BY starttime) AS int_sequence
FROM ventilation_type
WHERE ventilation_status NOT in ('None', 'SupplementalOxygen') AND ventilation_status IS NOT NULL
)
SELECT stay_id, inttime, int_type FROM iws WHERE int_sequence = 1
""")
stays_with_interventions.count("*")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│         9263 │
└──────────────┘

In [6]:
# Patients with index period
patient_with_index_period=duckdb.sql("""
WITH patient_with_intervention AS (
    SELECT isd.*, v.inttime, v.int_type FROM mimiciv_derived.icustay_detail AS isd
    LEFT OUTER JOIN stays_with_interventions AS v ON isd.stay_id = v.stay_id
    WHERE first_icu_stay AND first_hosp_stay
)
SELECT subject_id, stay_id, gender, race AS ethnicity, 
    admittime AS ip_starttime,
    GREATEST(admittime, LEAST( dischtime, inttime, admittime + interval '5 days')) AS ip_endtime,
    inttime, int_type
    FROM patient_with_intervention
""")
patient_with_index_period.df()                     

,subject_id,stay_id,gender,ethnicity,ip_starttime,ip_endtime,inttime,int_type
0,10001884,37510196,F,BLACK/AFRICAN AMERICAN,2131-01-07 20:39:00,2131-01-12 20:39:00,2131-01-19 18:40:00,InvasiveVent
1,10002155,33685454,F,WHITE,2129-08-04 12:44:00,2129-08-08 12:00:00,2129-08-08 12:00:00,HFNC
2,10002348,32610785,F,WHITE,2112-11-30 22:22:00,2112-12-05 02:45:00,2112-12-05 02:45:00,Tracheostomy
3,10002430,38392119,M,WHITE,2129-06-13 00:00:00,2129-06-13 07:00:00,2129-06-13 07:00:00,HFNC
4,10004401,32773003,M,WHITE,2144-01-23 07:58:00,2144-01-28 07:58:00,2144-02-03 08:00:00,Tracheostomy
...,...,...,...,...,...,...,...,...
50915,19975121,33599105,F,WHITE,2118-01-18 00:58:00,2118-01-21 08:00:00,2118-01-21 08:00:00,HFNC
50916,19984860,31082854,F,HISPANIC OR LATINO,2114-03-04 20:59:00,2114-03-09 00:15:00,2114-03-09 00:15:00,InvasiveVent
50917,19857813,36271025,F,HISPANIC/LATINO - PUERTO RICAN,2129-06-14 02:18:00,2129-06-14 09:00:00,2129-06-14 09:00:00,HFNC
50918,19858302,35257959,M,WHITE,2182-07-24 22:05:00,2182-07-25 11:00:00,2182-07-25 11:00:00,InvasiveVent


In [7]:
## VALIDATE: Patients with index period
# Unique patients (subject id)
assert duckdb.sql("SELECT subject_id, COUNT(*) FROM patient_with_index_period GROUP BY subject_id HAVING COUNT(*) > 1").fetchone() is None
# Unique stays (stay id)
assert duckdb.sql("SELECT stay_id, COUNT(*) FROM patient_with_index_period GROUP BY stay_id HAVING COUNT(*) > 1").fetchone() is None
# valid index period (ip_endtime >= ip_starttime)
assert duckdb.sql("SELECT * FROM patient_with_index_period WHERE ip_endtime < ip_starttime").fetchone() is None
# max lenth of index period is 5 days
assert duckdb.sql("SELECT * FROM patient_with_index_period WHERE ip_endtime - ip_starttime > interval '5 days'").fetchone() is None

In [8]:
# Valida nasal canula o2 flow
nasal_canula_o2_flow = duckdb.sql("""
WITH nc_o2 AS (
    SELECT *, LEAST(o2_flow, o2_flow_additional) AS o2_flow_nc FROM mimiciv_derived.oxygen_delivery WHERE o2_delivery_device_1 = 'Nasal cannula' AND o2_delivery_device_2 IS NULL
) 
SELECT subject_id, stay_id, charttime, o2_flow_nc AS o2_flow FROM nc_o2 WHERE o2_flow_nc <= 6
""")
nasal_canula_o2_flow

┌────────────┬──────────┬─────────────────────┬─────────┐
│ subject_id │ stay_id  │      charttime      │ o2_flow │
│   int32    │  int32   │      timestamp      │ double  │
├────────────┼──────────┼─────────────────────┼─────────┤
│   10000032 │ 39553978 │ 2180-07-23 14:20:00 │     2.0 │
│   10000032 │ 39553978 │ 2180-07-23 18:00:00 │     2.0 │
│   10000032 │ 39553978 │ 2180-07-23 20:00:00 │     2.0 │
│   10000980 │ 39765666 │ 2189-06-27 10:00:00 │     3.0 │
│   10001217 │ 37067082 │ 2157-11-20 19:47:00 │     2.0 │
│   10001217 │ 37067082 │ 2157-11-20 22:00:00 │     2.0 │
│   10001217 │ 37067082 │ 2157-11-21 00:00:00 │     2.0 │
│   10001217 │ 37067082 │ 2157-11-21 02:00:00 │     2.0 │
│   10001217 │ 37067082 │ 2157-11-21 08:00:00 │     2.0 │
│   10001217 │ 37067082 │ 2157-11-21 18:00:00 │     3.0 │
│       ·    │     ·    │          ·          │      ·  │
│       ·    │     ·    │          ·          │      ·  │
│       ·    │     ·    │          ·          │      ·  │
│   10269842 │

In [9]:
patient_with_o2_data = duckdb.sql("""
SELECT * FROM patient_with_index_period AS pwi 
WHERE EXISTS( SELECT 1 FROM nasal_canula_o2_flow AS nco WHERE pwi.stay_id = nco.stay_id AND nco.charttime BETWEEN pwi.ip_starttime AND pwi.ip_endtime)
""")
patient_with_o2_data

┌────────────┬──────────┬─────────┬───┬─────────────────────┬─────────────────────┬─────────────────┐
│ subject_id │ stay_id  │ gender  │ … │     ip_endtime      │       inttime       │    int_type     │
│   int32    │  int32   │ varchar │   │      timestamp      │      timestamp      │     varchar     │
├────────────┼──────────┼─────────┼───┼─────────────────────┼─────────────────────┼─────────────────┤
│   10103795 │ 31411464 │ M       │ … │ 2176-07-06 16:56:00 │ 2176-07-09 22:00:00 │ Tracheostomy    │
│   10119017 │ 31009936 │ M       │ … │ 2159-08-13 15:00:00 │ 2159-08-13 15:00:00 │ HFNC            │
│   10259667 │ 35629197 │ M       │ … │ 2171-04-21 00:00:00 │ 2171-04-21 00:00:00 │ NonInvasiveVent │
│   10005606 │ 38740124 │ M       │ … │ 2143-12-11 00:54:00 │ NULL                │ NULL            │
│   10026868 │ 31030669 │ M       │ … │ 2149-10-30 14:42:00 │ NULL                │ NULL            │
│   10027704 │ 36187473 │ M       │ … │ 2149-04-08 10:39:00 │ NULL                

In [10]:
# Initial cohort from the paper: 25,340
patient_with_o2_data.count("*")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        28942 │
└──────────────┘

In [11]:
patient_with_race = duckdb.sql("""
WITH patients AS (
    SELECT *,
    CASE
        WHEN ethnicity LIKE 'ASIAN%' THEN 'ASIAN'
        WHEN ethnicity LIKE 'BLACK%' THEN 'BLACK'
        WHEN ethnicity LIKE 'WHITE%' THEN 'WHITE'
        WHEN ethnicity LIKE 'HISPANIC%' THEN 'HISPANIC'
        END AS race_category
    FROM patient_with_o2_data)
SELECT subject_id, stay_id, race_category, gender, ip_starttime, ip_endtime  FROM patients WHERE race_category IS NOT NULL
""")
patient_with_race

┌────────────┬──────────┬───────────────┬─────────┬─────────────────────┬─────────────────────┐
│ subject_id │ stay_id  │ race_category │ gender  │    ip_starttime     │     ip_endtime      │
│   int32    │  int32   │    varchar    │ varchar │      timestamp      │      timestamp      │
├────────────┼──────────┼───────────────┼─────────┼─────────────────────┼─────────────────────┤
│   10103795 │ 31411464 │ WHITE         │ M       │ 2176-07-01 16:56:00 │ 2176-07-06 16:56:00 │
│   10119017 │ 31009936 │ ASIAN         │ M       │ 2159-08-10 23:55:00 │ 2159-08-13 15:00:00 │
│   10259667 │ 35629197 │ WHITE         │ M       │ 2171-04-18 12:35:00 │ 2171-04-21 00:00:00 │
│   10005606 │ 38740124 │ WHITE         │ M       │ 2143-12-06 00:54:00 │ 2143-12-11 00:54:00 │
│   10026868 │ 31030669 │ WHITE         │ M       │ 2149-10-25 14:42:00 │ 2149-10-30 14:42:00 │
│   10027704 │ 36187473 │ WHITE         │ M       │ 2149-04-03 10:39:00 │ 2149-04-08 10:39:00 │
│   10035787 │ 36110350 │ WHITE         

In [12]:
# Patients with documented rare/enthnicity by race
duckdb.sql("""
SELECT race_category, COUNT(*) FROM patient_with_race 
GROUP BY race_category
""")

┌───────────────┬──────────────┐
│ race_category │ count_star() │
│    varchar    │    int64     │
├───────────────┼──────────────┤
│ HISPANIC      │          915 │
│ BLACK         │         2259 │
│ WHITE         │        20496 │
│ ASIAN         │          738 │
└───────────────┴──────────────┘

In [13]:
reading_o2_flow = duckdb.sql("""
SELECT pwr.subject_id, od.charttime as chart_time, od.o2_flow FROM patient_with_index_period AS pwr
JOIN nasal_canula_o2_flow AS od ON  pwr.stay_id = od.stay_id
WHERE od.charttime BETWEEN pwr.ip_starttime AND pwr.ip_endtime AND od.o2_flow IS NOT NULL
ORDER BY pwr.subject_id, od.charttime
""")
reading_o2_flow

┌────────────┬─────────────────────┬─────────┐
│ subject_id │     chart_time      │ o2_flow │
│   int32    │      timestamp      │ double  │
├────────────┼─────────────────────┼─────────┤
│   10000032 │ 2180-07-23 14:20:00 │     2.0 │
│   10000032 │ 2180-07-23 18:00:00 │     2.0 │
│   10000032 │ 2180-07-23 20:00:00 │     2.0 │
│   10000980 │ 2189-06-27 10:00:00 │     3.0 │
│   10001217 │ 2157-11-20 19:47:00 │     2.0 │
│   10001217 │ 2157-11-20 22:00:00 │     2.0 │
│   10001217 │ 2157-11-21 00:00:00 │     2.0 │
│   10001217 │ 2157-11-21 02:00:00 │     2.0 │
│   10001217 │ 2157-11-21 08:00:00 │     2.0 │
│   10001217 │ 2157-11-21 18:00:00 │     3.0 │
│       ·    │          ·          │      ·  │
│       ·    │          ·          │      ·  │
│       ·    │          ·          │      ·  │
│   10524082 │ 2130-06-26 12:00:00 │     2.0 │
│   10524082 │ 2130-06-26 16:00:00 │     2.0 │
│   10524082 │ 2130-06-26 20:00:00 │     2.0 │
│   10524082 │ 2130-06-26 21:00:00 │     4.0 │
│   10524082 

### Sp02

In [14]:
reading_spo2 = duckdb.sql("""
SELECT pwr.subject_id, vs.charttime as chart_time, vs.spo2 FROM patient_with_index_period AS pwr
JOIN mimiciv_derived.vitalsign AS vs ON  pwr.stay_id = vs.stay_id
WHERE vs.charttime BETWEEN pwr.ip_starttime AND pwr.ip_endtime AND vs.spo2 IS NOT NULL
ORDER BY pwr.subject_id, vs.charttime
""")
reading_spo2

┌────────────┬─────────────────────┬────────┐
│ subject_id │     chart_time      │  spo2  │
│   int32    │      timestamp      │ double │
├────────────┼─────────────────────┼────────┤
│   10000032 │ 2180-07-23 14:13:00 │   98.0 │
│   10000032 │ 2180-07-23 14:30:00 │   97.0 │
│   10000032 │ 2180-07-23 15:00:00 │   97.0 │
│   10000032 │ 2180-07-23 16:00:00 │   94.0 │
│   10000032 │ 2180-07-23 17:00:00 │   95.0 │
│   10000032 │ 2180-07-23 18:00:00 │   95.0 │
│   10000032 │ 2180-07-23 19:00:00 │   98.0 │
│   10000032 │ 2180-07-23 20:00:00 │   99.0 │
│   10000032 │ 2180-07-23 21:00:00 │   95.0 │
│   10000032 │ 2180-07-23 22:00:00 │   95.0 │
│       ·    │          ·          │     ·  │
│       ·    │          ·          │     ·  │
│       ·    │          ·          │     ·  │
│   10035185 │ 2120-05-14 02:00:00 │   98.0 │
│   10035185 │ 2120-05-14 03:00:00 │   98.0 │
│   10035185 │ 2120-05-14 04:00:00 │   98.0 │
│   10035185 │ 2120-05-14 05:00:00 │   98.0 │
│   10035185 │ 2120-05-14 06:00:00

In [15]:
reading_so2 = duckdb.sql("""
SELECT pwr.subject_id, bg.charttime as chart_time, bg.so2 FROM patient_with_index_period AS pwr
JOIN mimiciv_derived.bg AS bg ON  pwr.subject_id = bg.subject_id
WHERE bg.charttime BETWEEN pwr.ip_starttime AND pwr.ip_endtime AND bg.so2 IS NOT NULL AND bg.specimen = 'ART.'
ORDER BY pwr.subject_id, bg.charttime
""")
reading_so2

┌────────────┬─────────────────────┬────────┐
│ subject_id │     chart_time      │  so2   │
│   int32    │      timestamp      │ double │
├────────────┼─────────────────────┼────────┤
│   10002013 │ 2160-05-18 16:03:00 │   99.0 │
│   10002013 │ 2160-05-18 17:42:00 │   96.0 │
│   10002013 │ 2160-05-18 21:32:00 │   97.0 │
│   10002155 │ 2129-08-05 07:48:00 │   94.0 │
│   10002443 │ 2183-10-18 02:35:00 │   90.0 │
│   10002760 │ 2141-04-20 16:10:00 │   97.0 │
│   10002760 │ 2141-04-20 18:18:00 │   99.0 │
│   10002760 │ 2141-04-21 02:04:00 │   99.0 │
│   10002760 │ 2141-04-21 04:03:00 │   99.0 │
│   10002760 │ 2141-04-21 06:04:00 │   98.0 │
│       ·    │          ·          │     ·  │
│       ·    │          ·          │     ·  │
│       ·    │          ·          │     ·  │
│   12412126 │ 2117-10-16 12:22:00 │   99.0 │
│   12412126 │ 2117-10-16 15:11:00 │   99.0 │
│   12412595 │ 2111-01-07 09:58:00 │   97.0 │
│   12412595 │ 2111-01-07 10:56:00 │   97.0 │
│   12413618 │ 2187-03-03 20:13:00

## Perform final extract

In [16]:
coh_subject = duckdb.sql("""
SELECT subject_id, gender, race_category FROM patient_with_race AS pwr
WHERE 
    EXISTS( SELECT 1 FROM reading_o2_flow AS rof WHERE pwr.subject_id = rof.subject_id)
    AND EXISTS( SELECT 1 FROM reading_spo2 AS rs WHERE pwr.subject_id = rs.subject_id)
    AND EXISTS( SELECT 1 FROM reading_so2 AS rso WHERE pwr.subject_id = rso.subject_id)
""")
coh_subject

┌────────────┬─────────┬───────────────┐
│ subject_id │ gender  │ race_category │
│   int32    │ varchar │    varchar    │
├────────────┼─────────┼───────────────┤
│   10746848 │ F       │ WHITE         │
│   11124859 │ M       │ WHITE         │
│   11236474 │ F       │ BLACK         │
│   12538134 │ F       │ ASIAN         │
│   13299872 │ F       │ WHITE         │
│   13678296 │ M       │ WHITE         │
│   14393652 │ M       │ WHITE         │
│   15583708 │ M       │ WHITE         │
│   16003671 │ M       │ HISPANIC      │
│   16979799 │ M       │ WHITE         │
│       ·    │ ·       │   ·           │
│       ·    │ ·       │   ·           │
│       ·    │ ·       │   ·           │
│   15588339 │ M       │ WHITE         │
│   15921116 │ M       │ WHITE         │
│   15942459 │ F       │ WHITE         │
│   15968932 │ M       │ WHITE         │
│   16297202 │ M       │ WHITE         │
│   17841897 │ M       │ WHITE         │
│   17902431 │ M       │ WHITE         │
│   18483313 │ F

In [17]:
duckdb.sql("""
SELECT race_category, COUNT(*) FROM coh_subject GROUP BY  race_category
""")

┌───────────────┬──────────────┐
│ race_category │ count_star() │
│    varchar    │    int64     │
├───────────────┼──────────────┤
│ HISPANIC      │          231 │
│ BLACK         │          482 │
│ WHITE         │         5904 │
│ ASIAN         │          207 │
└───────────────┴──────────────┘

In [18]:
coh_reading_o2_flow = duckdb.sql("""
SELECT * FROM reading_o2_flow AS rd WHERE EXISTS( SELECT 1 FROM coh_subject AS cs WHERE rd.subject_id = cs.subject_id)
""")
coh_reading_spo2 = duckdb.sql("""
SELECT * FROM reading_spo2 AS rd WHERE EXISTS( SELECT 1 FROM coh_subject AS cs WHERE rd.subject_id = cs.subject_id)
""")
coh_reading_so2 = duckdb.sql("""
SELECT * FROM reading_so2 AS rd WHERE EXISTS( SELECT 1 FROM coh_subject AS cs WHERE rd.subject_id = cs.subject_id)
""")

### Save the extreact to cvs

In [19]:
coh_subject.to_csv('../data/psql_mimic-2.2/subject.csv')
coh_reading_o2_flow.to_csv('../data/psql_mimic-2.2/reading_o2_flow.csv')
coh_reading_spo2.to_csv('../data/psql_mimic-2.2/reading_spo2.csv')
coh_reading_so2.to_csv('../data/psql_mimic-2.2/reading_so2.csv')